<div class="alert alert-block alert-info">
    
<h2>How to access this Jupyter notebook</h2>

* <b>Step 1</b>: Open a web browser,  and go to [this page](https://jupyter.warwick.ac.uk/module/CH274), 
* <b>Step 2</b>: Enter your SCRTP username and password and press the "Start Server" button.<br>
* <b>Step 3</b>: Wait (it could take a few minutes) until the blue box says "Jupyter notebook server running!". At that point, click on the weblink below said message.<br>
* <b>Step 4</b>: Select the Jupyter Notebook you want to work on. <i>Remember to make a copy of the orginal notebook</i> (which is read-only). To do so, in the toolbar on top of the notebook, select File and then Make a Copy <br>
* <b>Step 5</b>: You're all set! <br>
* <b>Step 6</b>: <font color="red">When you are done, remember to click the "Stop Server" button in the launcher web browser tab.</font> Please do, it's really quite important. <br>
<b> Remember: </b> You can access your copy of the Notebook at any time from any device off and on campus by going through the same steps on e.g. your laptop - all the changes you have made will be saved and synced! <br>

</div>

In [ ]:
#Import necessary modules
import numpy as np
import matplotlib.pyplot as plt
import psi4
from psi4 import properties
import ase
from ase.build import molecule
from ase.atoms import Atoms
from ase.visualize import view
from ase.visualize.plot import plot_atoms
from psi4_helper import geom_ase_to_psi4

# Correlated Methods and Excited States

## What is this exercise about?

In this workshop, we are going to apply multi-reference methods for the simulation of dissociation curves and excited states and compare their convergence and accuracy to single-reference methods.
You will investigate:
* Static and dynamic correlation
* how your choice of method can influence results
* basis set convergence of multi-reference and single-reference methods

# Introduction

In the previous exercise you have used the Hartree-Fock (HF) method to compute the energy of a system with different basis sets and carried out geometry optimizations as well as transition state searches. Here, we will try to calculate the dissociation curve of H$_2$ with HF, i.e., we will look at the following reaction:

Be$_2$ $\rightarrow$ 2Be

You will see that HF will fail to describe the dissociation of the beryllium molecule qualitatively due to strong static
correlation, which is important for systems with a lot of nearly degenerate orbitals close to the highest occupied molecular orbital (HOMO) and lowest occupied molecular orbital (LUMO). Here, for the bound state two electrons share the $\sigma$ orbital of the beryllium molecule. However, in the case of dissociation, there will be one electron in each 1s orbital of the two atoms. While the former case is closed-shell, the second is open shell.

More generally, in the dissociation limit our wavefunction ansatz has to account for the degeneracy of bonding and antibonding configurations. Since these configurations are comparably important to the total wavefunction, they have to be included in the calculation, making a multiconfigurational treatment necessary. Such quasi-degenerate configurations are not only important to correctly describe the breaking and formation of bonds, but also for excited states or transition states.

We will discuss different methods and their capability to describe the breaking and formation of bonds in molecules. You will see that a multi-reference wave function can account for static correlation and can provide a qualitatively correct picture of dissociated molecules.

In [ ]:
# prepare geometries
# we set the positions to 0 for atom 0 and to 2-4.0 for atom 2

all_atoms = []

positions=np.zeros((2,3))
atomtype = ["Be","Be"]
all_distances = np.array([2.0,2.2,2.4,2.5,2.6,2.8,3.0,3.5,4.0,4.5,5.0,6.0,7.0])
nsteps=len(all_distances)
for i,dist in enumerate(all_distances):
    positions=np.zeros((2,3))
    positions[1,2]=dist
    atoms = Atoms(atomtype,positions)
    all_atoms.append(atoms)
    #print(all_atoms[i].positions)



    
### Let's make a RHF Calculation for a set of Be$_2$ molecules with increasing bond length


In [ ]:
psi4.core.clean()
#psi4.core.reopen_outfile()
psi4.core.set_output_file('output.dat', False)
psi4.set_memory('1000 MB')
all_energies = []
wvfs_hf = []
##set geometry
#transform ase geometries to Psi4 input string
psi4.core.clean_options()
for i in range(nsteps):
    geom_input = geom_ase_to_psi4(all_atoms[i], charge=0, multiplicity=1,symmetry="c1")
    #initiate Psi4 molecule object
    h2 = psi4.geometry(geom_input)

    # Set computation options
    psi4.set_options({'basis': '3-21G',
                  'reference': 'rhf',
                  'scf_type':'df',
                  'e_convergence': 1e-8})

    E, wvf = psi4.energy('scf',return_wfn=True)
    all_energies.append(E)
    wvfs_hf.append(wvf)
    print(r'Total HF energy at {0:4.2f}A distance is {1:16.8f} Hartree'.format(all_atoms[i].positions[1,2],E))


In [ ]:
#%less output.dat

In [ ]:
plt.plot(all_distances,all_energies,label = "RHF")
plt.legend()

### MP2

In [ ]:
## Let's see if MP2 can give a better result

all_energies_mp2 = []
for i in range(nsteps):
    geom_input = geom_ase_to_psi4(all_atoms[i], charge=0, multiplicity=1)
    #initiate Psi4 molecule object
    h2 = psi4.geometry(geom_input)

    E, wvf_mp2 = psi4.energy('mp2',return_wfn=True)
    all_energies_mp2.append(E)
    print(r'Total MP2 energy at {0:4.2f}A distance is {1:16.8f} Hartree'.format(all_atoms[i].positions[1,2],E))


In [ ]:
min_mp2 = np.min(all_energies_mp2)
min_hf = np.min(all_energies)
plt.plot(all_distances,all_energies-min_hf,label="HF")
plt.plot(all_distances,all_energies_mp2-min_mp2,label="MP2",linestyle="--")
plt.legend()

As you can see, HF and MP2 both fail completely to describe the ground state potential along the dissociative coordinate of the Be$_2$ molecule, which should look more similar to the following curve taken from DOI: 10.1126/science.1174326: https://science.sciencemag.org/content/324/5934/1548

The experimentally measured well depth for Be2 is 9.45 kJ/mol (98 meV or 0.0036 Hartree)

</div>
<img src="Be.png" width="750"/>
</div>


Therefore, we will try to investigate the solution with <b> CASSCF <b>.

### CASSCF

#### Try to come up with a solution to the active space

Use the following options and fill in the ones that are left out, which are "frozen_docc" and "active".

"frozen_docc" specifies the orbitals that are inactive and doubly occupied. "active" specifies the orbitals above the "frozen_docc" block that will be included into the active space. For the other settings, have a look at the description of the DETCI module in the Psi4 manual.



    psi4.set_options({})
    psi4.set_options({
        'basis': '3-21G',
        'reference': 'rhf',
        'scf_type':'pk',
        'mcscf_algorithm':'ah',
        'qc_module': 'detci',
        'nat_orbs': True,
        'frozen_docc': ,
        'active':
    })


<div class="alert alert-block alert-info">

### Task

To choose the number of frozen orbitals and the number of active orbitals, you should first visualize the RHF orbitals and MO energies. How many orbitals should be included in the frozen_docc region?
Scan through the binding energy curve and study how the orbital energies change.
    
</div>

In [ ]:
########################
epsilons = np.zeros([nsteps,wvf.nmo()])
occs = np.zeros([nsteps,wvf.nmo()])
for i in range(nsteps):
    wvf=wvfs_hf[i]
    coeffs = wvf.Ca().to_array()
    epsilons[i,:] = wvf.epsilon_a().to_array()
    occs[i,:] = wvf.occupation_a().to_array()

print("Orbitals at 2.2 Angstrom")
for i,(o,e) in enumerate(zip(occs[3], epsilons[3])):
    print ('Molecular Orbital {0}, occupation: {1:4.2f}, energy: {2:8.3f}'
           .format(i,o*2,e))

In [ ]:
plt.plot(all_distances,epsilons[:,0],label="HOMO-3")
plt.plot(all_distances,epsilons[:,1],label="HOMO-2")
plt.plot(all_distances,epsilons[:,2],label="HOMO-1")
plt.plot(all_distances,epsilons[:,3],label="HOMO")
plt.plot(all_distances,epsilons[:,4],label="LUMO")
plt.plot(all_distances,epsilons[:,5],label="LUMO+1")
plt.plot(all_distances,epsilons[:,6],label="LUMO+2")
plt.plot(all_distances,epsilons[:,7],label="LUMO+3")
plt.plot(all_distances,epsilons[:,8],label="LUMO+4")
plt.plot(all_distances,epsilons[:,9],label="LUMO+5")
plt.plot(all_distances,epsilons[:,10],label="LUMO+6")
plt.plot(all_distances,epsilons[:,11],label="LUMO+7")

plt.legend(loc="lower right", bbox_to_anchor=[0,0,1.3,0.7])
plt.ylim(-5,0.5) # refine to -0.5, 0.5 to focus on valence electrons
plt.show()


We find that we can definitely include the 2 1s core electrons into the frozen doubly occupied region, but probably also the HOMO-1 (at least if we don't go beyond 7 Angstrom) so we set
    
    frozen_docc: [3]
    
We also find that HOMO-1 and HOMO become degenerate at large distances and we find that LUMO to LUMO+5 coalesce at large distance. We will need at least the HOMO-1, HOMO and some of the unoccupied states in the active space

<div class="alert alert-block alert-info">

### Task

Now run the CASSCF calculation with various different active space settings ranging from 3 to 8, 
    corresponding to CAS(4,3) to CAS(4,8),ie. 4 electrons in 3 orbitals or 4 electrons in 8 orbitals.
</div>

In [ ]:
casscf_tests=[]

active_set = [2,3,6,7]

for active in active_set:

    all_energies_casscf = []
    psi4.core.clean()
    psi4.core.clean_options()
    psi4.set_options({
    'basis': '3-21G',
    'reference': 'rhf',
    'scf_type':'pk',
    'mcscf_algorithm':'ah',
    'qc_module': 'detci',
    'nat_orbs': True,
    'frozen_docc': [3],
    'active': [active]
})
    psi4.core.set_output_file('Be.out', False)
    for i in range(nsteps):
        geom_input = geom_ase_to_psi4(all_atoms[i], charge=0, multiplicity=1)
        #initiate Psi4 molecule object
        be2 = psi4.geometry(geom_input)
        E, wvf = psi4.energy('casscf',return_wfn=True)
        all_energies_casscf.append(E)
        print(r'Total CASSCF energy at {0:4.2f}A distance is {1:16.8f} Hartree'.format(all_atoms[i].positions[1,2],E))

    casscf_tests.append(all_energies_casscf)

In [ ]:
#%less Be.out

In [ ]:
plt.plot(all_distances,all_energies-min_hf,label="HF")
plt.plot(all_distances,all_energies_mp2-min_mp2,label="MP2",linestyle="--")

for i, (all_energies_casscf, cas) in enumerate(zip(casscf_tests,active_set)):
    min_casscf = np.min(all_energies_casscf)
    plt.plot(all_distances,all_energies_casscf-min_casscf,label="CAS(2,{0})".format(cas,linestyle="-."))
    
plt.ylim(-0.0001, 0.05)
plt.legend()


CASSCF seems to converge to the right equilibrium distance, but the binding energy is off. Let's bring in some dynamical correlation.

### First up: DFT

In [ ]:
all_energies_dft = []
psi4.core.clean()
psi4.core.clean_options()
psi4.set_options({
    'basis': '3-21G',
    'scf_type':'df',
    'guess': 'sad'
})
psi4.core.set_output_file('BeDFT.out', False)
wvf_dft = []
for i in range(nsteps):
    geom_input = geom_ase_to_psi4(all_atoms[i], charge=0, multiplicity=1)
    #initiate Psi4 molecule object
    be2 = psi4.geometry(geom_input)
    E, wvf = psi4.energy('b3lyp',return_wfn=True)
    wvf_dft.append(wvf)
    all_energies_dft.append(E)
    print(r'Total DFT energy at {0:4.2f}A distance is {1:16.8f} Hartree'.format(all_atoms[i].positions[1,2],E))


In [ ]:
min_dft=np.min(all_energies_dft)

plt.plot(all_distances,all_energies-min_hf,label="HF")
plt.plot(all_distances,all_energies_mp2-min_mp2,label="MP2",linestyle="--")
plt.plot(all_distances,all_energies_casscf-min_casscf,label="CAS(2,7)",linestyle="-.")
plt.plot(all_distances,all_energies_dft-min_dft,label="DFT/B3LYP",linestyle="-.")
plt.legend()


## Let's bring in dynamical correlation with Multireference Coupled Cluser 

In [ ]:
mrcc_tests=[]

active_set = [2]

for active in active_set:

    all_energies_mrcc = []
    psi4.core.clean()
    psi4.core.clean_options()
    psi4.set_options({
    'basis': '3-21G',
    'reference': 'rhf',
    'scf_type':'pk',
    'mcscf_algorithm':'ah',
    #'qc_module': 'detci',
    'nat_orbs': True,
    'frozen_docc': [3], #not part of active space
    'restricted_docc': [0], # part of active space but always occupied
    'active': [active], # active MOs
    'frozen_uocc': [0], # frozen virtual MOs
    'corr_multp': 1,
    'follow_root': 1,
    'maxiter': 200,
    'corr_wfn': 'ccsd',
})
    psi4.core.set_output_file('BeMRCC.out', False)
    for i in range(nsteps):
        geom_input = geom_ase_to_psi4(all_atoms[i], charge=0, multiplicity=1)
        #initiate Psi4 molecule object
        be2 = psi4.geometry(geom_input)
        E, wvf = psi4.energy('psimrcc',return_wfn=True)
        all_energies_mrcc.append(E)
        print(r'Total MRCC energy at {0:4.2f}A distance is {1:16.8f} Hartree'.format(all_atoms[i].positions[1,2],E))

    mrcc_tests.append(all_energies_mrcc)

In [ ]:
min_mrcc=np.min(all_energies_mrcc)

plt.plot(all_distances,all_energies-min_hf,label="HF")
plt.plot(all_distances,all_energies_mp2-min_mp2,label="MP2",linestyle="--")
plt.plot(all_distances,all_energies_casscf-min_casscf,label="CAS(2,7)",linestyle="-.")
plt.plot(all_distances,all_energies_dft-min_dft,label="DFT/B3LYP",linestyle="-.")
plt.plot(all_distances,all_energies_mrcc-min_mrcc,label="MRCCSD(2,3)",linestyle="-.")
plt.legend()
plt.ylim(-0.001,0.01)


Note the difference in the binding energy curves between DFT and MRCC

## Group Work: Converging the binding energy of Be$_2$

With each method the basis set convergence of the interaction energy should be investigated. Calculate the difference in energy between the experimentally determined distance of 2.5 Angstrom distance and at 7 Angstrom distance. 
Calculate this energy difference for three different basis sets
* def2-SVP, 
* def2-TZVP, 
* def2-QZVP basis set
    
Plot the results (energies vs. 1,2,3) and evaluate the timings (look into the output files) and let us know whenever you are finished. In the end, we will compare the results and discuss the differences of each method.

The methods that should be tested are:
* RHF
* MP2
* DFT/PBE
* DFT/PBE0
* DFT/B3LYP
* CCSD
* CASSCF(2,3)
* CASSCF(2,7)
* MRCCSD(2,3) [corr_wfn: "CCSD"]
* MRPT2(2,3) [corr_wfn: "PT2"]


In [ ]:
all_atoms = []
nsteps=20
positions=np.zeros((2,3))
atomtype = ["Be","Be"]
all_distances = np.linspace(2.5,7.,2)
for i,dist in enumerate(all_distances):
    positions=np.zeros((2,3))
    positions[1,2]=dist
    atoms = Atoms(atomtype,positions)
    all_atoms.append(atoms)


all_energies_casscf = []
psi4.core.clean()
psi4.core.clean_options()
psi4.set_options({
    'basis': 'def2-SVP',
    'reference': 'rhf',
    #'scf_type':'pk',
    #'mcscf_algorithm':'ah',
    #'qc_module': 'detci',
    #'nat_orbs': True,
    #'frozen_docc': [2],
    #'active': [active]
})

energies = []

psi4.core.set_output_file('Be_interaction.out', False)
for i in range(2):
    geom_input = geom_ase_to_psi4(all_atoms[i], charge=0, multiplicity=1)
#initiate Psi4 molecule object
    be2 = psi4.geometry(geom_input)
    E, wvf = psi4.energy('pbe',return_wfn=True)
    energies.append(E)
    print(r'Total energy at {0:4.2f}A distance is {1:16.8f} Hartree'.format(all_atoms[i].positions[1,2],E))

print(energies[1]-energies[0], 'Hartree (exp: 0.0036 Hartree)')

## References
1. [[Szabo:1996](http://store.doverpublications.com/0486691861.html)] A. Szabo and N. S. Ostlund, *Modern Quantum Chemistry*, Introduction to Advanced Electronic Structure Theory. Courier Corporation, 1996.
2. [[Levine:2000](https://books.google.com/books?id=80RpQgAACAAJ&dq=levine%20quantum%20chemistry%205th%20edition&source=gbs_book_other_versions)] I. N. Levine, *Quantum Chemistry*. Prentice-Hall, New Jersey, 5th edition, 2000.
3. [[Helgaker:2000](https://books.google.com/books?id=lNVLBAAAQBAJ&pg=PT1067&dq=helgaker+molecular+electronic+structure+theory&hl=en&sa=X&ved=0ahUKEwj37I7MkofUAhWG5SYKHaoPAAkQ6AEIKDAA#v=onepage&q=helgaker%20molecular%20electronic%20structure%20theory&f=false)] T. Helgaker, P. Jorgensen, and J. Olsen, *Molecular Electronic Structure Theory*, John Wiley & Sons Inc, 2000.